# Laboratório 06: Modelo Vetorial

Antes de responder as perguntas deste laboratório, precisamos importar as extensões que usaremos no decorrer da atividade, tais como pandas, numpy e nltk.


*  **Importação das extensões necessárias** 

In [1]:
import csv
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Questão 1

**Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade.**



*   **Leitura do CSV**

In [0]:
colecao = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_06/master/results.csv')
documentos = colecao['text']

*   **Criação dos índices**

In [0]:
tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indices = {}
n = 0

for texto in documentos:
  palavras = [palavra for palavra in tokenizador.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopwords and len(palavra) >= 3]  
  n += 1
  for t in palavras:
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(n)
    
for elemento in indices.items():
  d = dict(collections.Counter(elemento[1]))
  indices[elemento[0]] = list(d.items())

Agora, temos o índice com o novo conjunto de dados da questão. 

Exemplo:

In [4]:
tabela_indices = pd.DataFrame()
tabela_indices['Palavras'] = indices.keys()
tabela_indices['Documentos e frequências'] = indices.values()

tabela_indices.head(10)

,Palavras,Documentos e frequências
0,juíza,"[(1, 2), (2, 1)]"
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ..."
2,ivani,"[(1, 1), (2, 1)]"
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 2), (26, 1), (73..."
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78..."
5,brasília,"[(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (4..."
6,proibiu,"[(1, 1), (2, 1), (119, 1), (162, 1)]"
7,caráter,"[(1, 1), (15, 1), (36, 1), (60, 1), (89, 1), (..."
8,liminar,"[(1, 1), (2, 3), (119, 1), (217, 1)]"
9,nesta,"[(1, 2), (3, 1), (4, 1), (8, 1), (21, 1), (22,..."


# Questão 2

**Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário.**

Primeiro, faremos uma função que retorna o IDF do termo passado dado o dicionário.

In [0]:
def calculaIDF(m, k):
  valorIDF = round(np.log((m+1) / k), 2)
  return valorIDF

Agora, aplicaremos o IDF a cada palavra do ranking, adicionando o IDF ao dicionário em cada termo.

In [0]:
m = colecao.text.count()

for palavra in indices:
  k = len(indices[palavra])
  idf = calculaIDF(m, k)
  indices[palavra].append(idf)

Exemplo:

In [7]:
tabela_indices['IDF'] = [indice[-1] for indice in indices.values()]

tabela_indices.head(10)

,Palavras,Documentos e frequências,IDF
0,juíza,"[(1, 2), (2, 1), 4.83]",4.83
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ...",1.63
2,ivani,"[(1, 1), (2, 1), 4.83]",4.83
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 2), (26, 1), (73...",3.04
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78...",2.53
5,brasília,"[(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (4...",2.19
6,proibiu,"[(1, 1), (2, 1), (119, 1), (162, 1), 4.14]",4.14
7,caráter,"[(1, 1), (15, 1), (36, 1), (60, 1), (89, 1), (...",3.22
8,liminar,"[(1, 1), (2, 3), (119, 1), (217, 1), 4.14]",4.14
9,nesta,"[(1, 2), (3, 1), (4, 1), (8, 1), (21, 1), (22,...",0.91


# Questão 3

**Implementar as seguintes versões do modelo vetorial:**

**1. Representação binária**

In [0]:
def modeloVetorialBinario(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    pontos += (token in tokens_documento)
 
  return pontos

**2. TF**

In [0]:
def modeloVetorialTF(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    pontos += tokens_documento.count(token)
  
  return pontos

**3. TF-IDF**

In [0]:
def modeloVetorialTFIDF(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    cwd = tokens_documento.count(token)
    pontos += cwd * indices[token][-1]
  
  return round(pontos, 2)

**4. BM25**

In [0]:
def modeloVetorialBM25(consulta, documento, n):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  palavras = [palavra for palavra in tokens_consulta if palavra in tokens_documento]
    
  for palavra in palavras:
    cwd = tokens_documento.count(palavra)
    dfw = len(indices[palavra][:-1])
    pontos += (((k+1) * cwd) / (cwd + k)) * np.log10(((m+1) / dfw))
  
  return round(pontos, 2)

# Questão 4

**Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta.**

Como primeiro passo, definiremos as 3 consultas para fazer o experimento.

In [0]:
consultas = ['marcos pontes', 'jair bolsonaro', 'futebol']

Agora, criaremos a função que retorna os top-5 documentos mais similares às consultas.

In [0]:
def top5Documentos(consulta):
  n = 0
  d_binaria = []
  d_tf = []
  d_tfidf = []
  d_bm25 = []
  for documento in colecao.text:
    documento = documento.lower()
    n += 1
    bisect.insort(d_binaria, (modeloVetorialBinario(consulta, documento), n))
    bisect.insort(d_tf, (modeloVetorialTF(consulta, documento), n))
    bisect.insort(d_tfidf, (modeloVetorialTFIDF(consulta, documento), n))
    bisect.insort(d_bm25, (modeloVetorialBM25(consulta, documento, 10), n))
  
  d_binaria.reverse()
  d_tf.reverse()
  d_tfidf.reverse()
  d_bm25.reverse()
  
  return d_binaria[:5], d_tf[:5], d_tfidf[:5], d_bm25[:5]

In [0]:
binario_top5 = ['','','']
tf_top5 = ['','','']
tfidf_top5 = ['','','']
bm25_top5 = ['','','']

binario_top5[0], tf_top5[0], tfidf_top5[0], bm25_top5[0] = top5Documentos(consultas[0])
binario_top5[1], tf_top5[1], tfidf_top5[1], bm25_top5[1] = top5Documentos(consultas[1])
binario_top5[2], tf_top5[2], tfidf_top5[2], bm25_top5[2] = top5Documentos(consultas[2])

Por fim, montaremos a tabela para mostrar os dados calculados.

In [15]:
tabela_q4 = pd.DataFrame()

tabela_q4['Consulta'] = consultas
tabela_q4['Binário'] = binario_top5
tabela_q4['TF'] = tf_top5
tabela_q4['TF-IDF'] = tfidf_top5
tabela_q4['BM25'] = bm25_top5

tabela_q4.index += 1
tabela_q4

,Consulta,Binário,TF,TF-IDF,BM25
1,marcos pontes,"[(2, 42), (1, 235), (1, 173), (1, 101), (1, 96)]","[(6, 42), (3, 235), (1, 173), (1, 101), (1, 96)]","[(26.52, 42), (9.96, 235), (3.32, 173), (3.32,...","[(5.76, 42), (2.17, 235), (1.44, 173), (1.44, ..."
2,jair bolsonaro,"[(2, 238), (2, 237), (2, 231), (2, 224), (2, 2...","[(41, 207), (37, 151), (31, 166), (19, 19), (1...","[(67.98, 207), (55.14, 151), (42.18, 166), (25...","[(3.16, 207), (2.97, 151), (2.23, 166), (2.17,..."
3,futebol,"[(1, 243), (1, 242), (1, 212), (1, 189), (1, 1...","[(7, 242), (7, 47), (6, 115), (5, 118), (3, 212)]","[(17.36, 242), (17.36, 47), (14.88, 115), (12....","[(1.88, 242), (1.88, 47), (1.84, 115), (1.79, ..."


# Questão 5

**Compare os resultados encontrados e responda.**

**1. Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.**

Para melhor embasar nossa resposta, vamos analisar os documentos que serão retornados, incluindo seus títulos.

In [0]:
def getTituloDocumento(documentos):
  return [colecao.title[documento] for documento in documentos]

def getPontuacaoTop1(tops):
  return [top[0][0] for top in tops]

def getDocumentoTop1(tops):
  return [top[0][1] for top in tops]

In [0]:
pontos_top5_binario = getPontuacaoTop1(binario_top5)
pontos_top5_tf = getPontuacaoTop1(tf_top5)
pontos_top5_tfidf = getPontuacaoTop1(tfidf_top5)
pontos_top5_bm25 = getPontuacaoTop1(bm25_top5)

documento_top5_binario = getDocumentoTop1(binario_top5)
documento_top5_tf = getDocumentoTop1(tf_top5)
documento_top5_tfidf = getDocumentoTop1(tfidf_top5)
documento_top5_bm25 = getDocumentoTop1(bm25_top5)

titulos_binario = getTituloDocumento(documento_top5_binario)
titulos_tf = getTituloDocumento(documento_top5_tf)
titulos_tfidf = getTituloDocumento(documento_top5_tfidf)
titulos_bm25 = getTituloDocumento(documento_top5_bm25)

Agora, construiremos a tabela para cada modelo com os dados acima.

a) Representação binária

In [18]:
tabela_q5_binaria = pd.DataFrame()
tabela_q5_binaria['Consulta'] = consultas
tabela_q5_binaria['Documento'] = documento_top5_binario
tabela_q5_binaria['Título'] = titulos_binario
tabela_q5_binaria['Pontuação'] = pontos_top5_binario

tabela_q5_binaria.index += 1
tabela_q5_binaria

,Consulta,Documento,Título,Pontuação
1,marcos pontes,42,“Mostre o que as loucas podem fazer”: o anúnci...,2
2,jair bolsonaro,238,Saúde mental dos estudantes mais um desafio p...,2
3,futebol,243,Nanga Parbat a obsessão com a ‘montanha assas...,1


b) TF

In [19]:
tabela_q5_tf = pd.DataFrame()
tabela_q5_tf['Consulta'] = consultas
tabela_q5_tf['Documento'] = documento_top5_tf
tabela_q5_tf['Título'] = titulos_tf
tabela_q5_tf['Pontuação'] = pontos_top5_tf

tabela_q5_tf.index += 1
tabela_q5_tf

,Consulta,Documento,Título,Pontuação
1,marcos pontes,42,“Mostre o que as loucas podem fazer”: o anúnci...,6
2,jair bolsonaro,207,Bolsonaro escancara cadáver insepulto da ditad...,41
3,futebol,242,A fortuna blindada de Cristiano Ronaldo,7


c) TF-IDF

In [20]:
tabela_q5_tfidf = pd.DataFrame()
tabela_q5_tfidf['Consulta'] = consultas
tabela_q5_tfidf['Documento'] = documento_top5_tfidf
tabela_q5_tfidf['Título'] = titulos_tfidf
tabela_q5_tfidf['Pontuação'] = pontos_top5_tfidf

tabela_q5_tfidf.index += 1
tabela_q5_tfidf

,Consulta,Documento,Título,Pontuação
1,marcos pontes,42,“Mostre o que as loucas podem fazer”: o anúnci...,26.52
2,jair bolsonaro,207,Bolsonaro escancara cadáver insepulto da ditad...,67.98
3,futebol,242,A fortuna blindada de Cristiano Ronaldo,17.36


d) BM25

In [21]:
tabela_q5_bm25 = pd.DataFrame()
tabela_q5_bm25['Consulta'] = consultas
tabela_q5_bm25['Documento'] = documento_top5_bm25
tabela_q5_bm25['Título'] = titulos_bm25
tabela_q5_bm25['Pontuação'] = pontos_top5_bm25

tabela_q5_bm25.index += 1
tabela_q5_bm25

,Consulta,Documento,Título,Pontuação
1,marcos pontes,42,“Mostre o que as loucas podem fazer”: o anúnci...,5.76
2,jair bolsonaro,207,Bolsonaro escancara cadáver insepulto da ditad...,3.16
3,futebol,242,A fortuna blindada de Cristiano Ronaldo,1.88


Comparando as tabelas, percebemos os títulos de cada documento mostrado como melhor resultado para a consulta. Sendo assim, podemos perceber que o TF-IDF e o BM25 apresentam resultados muito parecidos, além de aparentarem ser os melhores para os testes feitos. 

Para as consultas "marcos pontes", "jair bolsonaro" e "futebol" que foram realizadas, ambos os algoritmos apresentaram o mesmo documento. No entanto, pelo grau de complexidade da implementação dos algoritmos, o TF-IDF merece mais destaque pois, além de mais simples comparado ao BM25, nos proporciona um resultado bastante satisfatório.

**2. Calcule e reporte o overlap par-a-par entre os resultados de cada modelo.**

Primeiramente, definimos uma função calcula o índice de Jaccard.

In [0]:
def indiceJaccard(a, b):
  na = len(a)
  nb = len(b)
  nab = len([elem for elem in a if elem in b])
  expressao = na + nb - nab
  if expressao != 0:
    valorJaccard = nab / expressao
  else:
    valorJaccard = 0
  return valorJaccard

Agora, iremos comparar os índices pela tabela abaixo.

In [23]:
todos_os_documentos = []
todos_os_documentos.append(documento_top5_binario)
todos_os_documentos.append(documento_top5_bm25)
todos_os_documentos.append(documento_top5_tf)
todos_os_documentos.append(documento_top5_tfidf)

matriz = [['Modelo', 'Binário', 'TF', 'TF-IDF', 'BM25']]
titulos = matriz
for i in range(len(todos_os_documentos)):
  linha = []
  for j in range(len(todos_os_documentos)):
    jaccard = indiceJaccard(todos_os_documentos[i], todos_os_documentos[j])
    linha.append(round(jaccard, 2))
  matriz.append(linha)
for i in range(1, len(matriz)):
  matriz[i].insert(0, matriz[0][i])
  
pd.DataFrame(matriz)

,0,1,2,3,4
0,Modelo,Binário,TF,TF-IDF,BM25
1,Binário,1,0.2,0.2,0.2
2,TF,0.2,1,1,1
3,TF-IDF,0.2,1,1,1
4,BM25,0.2,1,1,1


De acordo com o indíce de Jaccard, percebemos que a discrepância do modelo binário é bem nítida, pois ele não possui similaridade com nenhum outro modelo. Entre os modelos TF e BM25, ocorre o valor máximo para o indíce de Jaccard, indicando assim a igualdade entre os dois modelos, fazendo com que confirmemos a escolha do TF-IDF como o modelo mais satisfatório. Já que, de acordo com os resultados da questão anterior, percebemos que o modelo TF apresenta resultados que não são satisfatórios para algumas consultas.